# Regular Expressions
Regular expressions, dubbed RegEx, is a sequence of characters that form a search pattern. Commonly used when extracting data from text or working with string data. RegEx can be used to check if a string matches or contains a specified pattern.<br>
Python's built in RegEx package is called `re`.

In [1]:
import re

In [2]:
txt = "Scientific Programming in Python"
x = re.search("P",txt)
print(x)


<re.Match object; span=(11, 12), match='P'>


In [3]:
print(x.start(), x.end())

11 12


In [4]:
y = re.search("Z",txt)
print(y)

None


In [5]:
z=re.findall("P",txt)
print(z)

['P', 'P']


In [6]:
z=re.findall("Z",txt)
print(z)

[]


In [7]:
s = re.sub("i","e",txt,2)
print(s)

Sceentefic Programming in Python


In [8]:
print(txt)
m = re.finditer(re.compile("i"),txt)
for i in m:
    print(i)

Scientific Programming in Python
<re.Match object; span=(2, 3), match='i'>
<re.Match object; span=(6, 7), match='i'>
<re.Match object; span=(8, 9), match='i'>
<re.Match object; span=(19, 20), match='i'>
<re.Match object; span=(23, 24), match='i'>


In [9]:
html = """<li id="menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746"><a href="https://techcrunch.com/mobile/">Mobile</a></li>
<li id="menu-item-899747" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899747"><a href="https://techcrunch.com/gadgets/">Gadgets</a></li>
<li id="menu-item-899748" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899748"><a href="https://techcrunch.com/enterprise/">Enterprise</a></li>
<li id="menu-item-899749" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899749"><a href="https://techcrunch.com/social/">Social</a></li>
<li id="menu-item-899750" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899750"><a href="https://techcrunch.com/europe/">Europe</a></li>
<li id="menu-item-901944" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-901944"><a href="/asia">Asia</a></li>
<li id="menu-item-1266871" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-1266871"><a href="https://techcrunch.com/crunch-network/">Crunch Network</a></li>
<li id="menu-item-1210304" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-1210304"><a href="https://techcrunch.com/unicorn-leaderboard/">Unicorn Leaderboard</a></li>
<li id="menu-item-1242803" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-1242803"><a href="https://techcrunch.com/gifts/">Gift Guides</a></li>"""

In [10]:
html

'<li id="menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746"><a href="https://techcrunch.com/mobile/">Mobile</a></li>\n<li id="menu-item-899747" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899747"><a href="https://techcrunch.com/gadgets/">Gadgets</a></li>\n<li id="menu-item-899748" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899748"><a href="https://techcrunch.com/enterprise/">Enterprise</a></li>\n<li id="menu-item-899749" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899749"><a href="https://techcrunch.com/social/">Social</a></li>\n<li id="menu-item-899750" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899750"><a href="https://techcrunch.com/europe/">Europe</a></li>\n<li id="menu-item-90

In [11]:
anchors = re.finditer("<a.*>",html)
for a in anchors:
    print(a.group())

<a href="https://techcrunch.com/mobile/">Mobile</a></li>
<a href="https://techcrunch.com/gadgets/">Gadgets</a></li>
<a href="https://techcrunch.com/enterprise/">Enterprise</a></li>
<a href="https://techcrunch.com/social/">Social</a></li>
<a href="https://techcrunch.com/europe/">Europe</a></li>
<a href="/asia">Asia</a></li>
<a href="https://techcrunch.com/crunch-network/">Crunch Network</a></li>
<a href="https://techcrunch.com/unicorn-leaderboard/">Unicorn Leaderboard</a></li>
<a href="https://techcrunch.com/gifts/">Gift Guides</a></li>


#### RegEx is greedy and will keep searching for a match until it reaches the end of a line. We need so be very clear on exactly which pattern we wish to match.

In [12]:
anchors = re.finditer("<a.*?>",html)
for a in anchors:
    print(a.group())

<a href="https://techcrunch.com/mobile/">
<a href="https://techcrunch.com/gadgets/">
<a href="https://techcrunch.com/enterprise/">
<a href="https://techcrunch.com/social/">
<a href="https://techcrunch.com/europe/">
<a href="/asia">
<a href="https://techcrunch.com/crunch-network/">
<a href="https://techcrunch.com/unicorn-leaderboard/">
<a href="https://techcrunch.com/gifts/">


###### here we search for a `<a` followed by any number of characters up until the first `>`

In [49]:
html

'<li id="menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746"><a href="https://techcrunch.com/mobile/">Mobile</a></li>\n<li id="menu-item-899747" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899747"><a href="https://techcrunch.com/gadgets/">Gadgets</a></li>\n<li id="menu-item-899748" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899748"><a href="https://techcrunch.com/enterprise/">Enterprise</a></li>\n<li id="menu-item-899749" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899749"><a href="https://techcrunch.com/social/">Social</a></li>\n<li id="menu-item-899750" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899750"><a href="https://techcrunch.com/europe/">Europe</a></li>\n<li id="menu-item-90

In [13]:
menu = re.finditer('menu(-[^\s"]*)[\s"]',html) #\s = whitespaces (\t, \r, \n, space characters)
for m in menu:
    print(m.group(1))

-item-899746
-item
-item-type-taxonomy
-item-object-category
-item-899746
-item-899747
-item
-item-type-taxonomy
-item-object-category
-parent
-item-899747
-item-899748
-item
-item-type-taxonomy
-item-object-category
-item-899748
-item-899749
-item
-item-type-taxonomy
-item-object-category
-item-899749
-item-899750
-item
-item-type-taxonomy
-item-object-category
-parent
-item-899750
-item-901944
-item
-item-type-custom
-item-object-custom
-item-901944
-item-1266871
-item
-item-type-post_type
-item-object-page
-item-1266871
-item-1210304
-item
-item-type-post_type
-item-object-page
-item-1210304
-item-1242803
-item
-item-type-post_type
-item-object-page
-item-1242803


In [14]:
menu = re.finditer('menu(-[^\s"]*)-(?P<num>\d+)',html)
for m in menu:
    print(m.group("num"))

899746
899746
899747
899747
899748
899748
899749
899749
899750
899750
901944
901944
1266871
1266871
1210304
1210304
1242803
1242803


In [42]:
menu = re.finditer(r'((menu(-[^\s"]*)-(?P<num>\d+)).*\2)',html)
results = []
for m in menu:
    results.append(m)
    print(m.groups())

('menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746', 'menu-item-899746', '-item', '899746')
('menu-item-899747" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899747', 'menu-item-899747', '-item', '899747')
('menu-item-899748" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899748', 'menu-item-899748', '-item', '899748')
('menu-item-899749" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899749', 'menu-item-899749', '-item', '899749')
('menu-item-899750" class="menu-item menu-item-type-taxonomy menu-item-object-category current-post-ancestor current-menu-parent current-post-parent menu-item-899750', 'menu-item-899750', '-item', '899750')
('menu-item-901944" class="menu-item menu-item-type-custom menu-item-object-custom menu-item-901944', 'menu-item-901944', '-item', '901944')
('menu-ite

In [48]:
print(results[0].groups())
print(results[0].group()) # default is 0 - the entire pattern match
print(results[0].group(0)) # group 0 is the entire pattern match
print(results[0].group(1)) # the first user defined group in parentheses


('menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746', 'menu-item-899746', '-item', '899746')
menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746
menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746
menu-item-899746" class="menu-item menu-item-type-taxonomy menu-item-object-category menu-item-899746


In [16]:
import csv

In [17]:
with open("sea-ice.csv") as file:
    reader = csv.DictReader(file)
    for i,entry in enumerate(reader):
        print(dict(entry))
        if (i>9):
            break

{'     Extent': '10.231', ' Source Data': " ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/arc/19781026-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'     Extent': '10.42', ' Source Data': " ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/r-19781028-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'     Extent': '10.557', ' Source Data': " ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nt_19781030__n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'     Extent': '10.67', ' Source Data': " ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/d/19781101-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'     Extent': '10.777', ' Source Data': " ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nd/19781103_n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'     Exten

In [18]:
data=[]
with open("sea-ice.csv") as file:
    reader = csv.DictReader(file)
    for entry in reader:
        tmp={}
        for key in entry.keys():
            tmp[key.strip()]=entry[key].strip()
        tmp['Extent']=float(tmp['Extent'])
        data.append(tmp)
for d in data[0:9]:
    print(d)
for d in data[-10:]:
    print(d)

{'Extent': 10.231, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/arc/19781026-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'Extent': 10.42, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/r-19781028-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'Extent': 10.557, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nt_19781030__n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'Extent': 10.67, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/d/19781101-n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'Extent': 10.777, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nd/19781103_n07_v1.1_n.bin']", 'hemisphere': 'north'}
{'Extent': 10.968, 'Source Data': "['ftp://sidads.colorad

In [50]:
print(data[-1])

{'Extent': 10.436, 'Source Data': "['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0081_nrt_nasateam_seaice/south/nd/20190530--f18_nrt_s.bin', 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0081_nrt_nasateam_seaice/south/nd/20190531--f18_nrt_s.bin', 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0081_nrt_nasateam_seaice/south/nd/20190601--f18_nrt_s.bin']", 'hemisphere': 'south'}


#### We would like to create a new data set which containst the date, extent, dataset name and hemisphere for each row
We will start by turning "Source Data" into a python list. Lists are valid JSON objects, so we will use the `json.loads()` function

In [19]:
import json

In [20]:
s = data[0]["Source Data"].replace("'",'"')
print(type(s),s)
j = json.loads(s)
print(type(j), j)

<class 'str'> ["ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/arc/19781026-n07_v1.1_n.bin"]
<class 'list'> ['ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/arc/19781026-n07_v1.1_n.bin']


In [21]:
def parseList(s):
    return json.loads(s.replace("'",'"'))

In [22]:
len(data)

26354

In [23]:
newData = []
for i,d in enumerate(data):
    sources = parseList(d["Source Data"])
    for src in sources:
        tmp = dict(d)
        del tmp["Source Data"]
        tmp["src"] = src
        newData.append(tmp)

In [24]:
len(newData)

28400

In [25]:
for d in newData[0:9]:
    print(d)
for d in newData[-10:]:
    print(d)

{'Extent': 10.231, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/arc/19781026-n07_v1.1_n.bin'}
{'Extent': 10.42, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/r-19781028-n07_v1.1_n.bin'}
{'Extent': 10.557, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nt_19781030__n07_v1.1_n.bin'}
{'Extent': 10.67, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/d/19781101-n07_v1.1_n.bin'}
{'Extent': 10.777, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seaice/final-gsfc/north/daily/1978/nd/19781103_n07_v1.1_n.bin'}
{'Extent': 10.968, 'hemisphere': 'north', 'src': 'ftp://sidads.colorado.edu/pub/DATASETS/nsidc0051_gsfc_nasateam_seai

#### We will create a RegEx to extract the dataset name and date from each src

In [26]:
pDataset = "(nsidc\d*)"
pDate = "(\d{8})[-_]"

## The datetime object

In [27]:
import datetime

x = datetime.datetime.now()

print(x.year)
print(x.strftime("%A")) #print day of week
print(x.strftime("%B")) #print month

2020
Tuesday
December


*The datetime constructor accepts parameters in the following order: year, month, day, hour, minute, second, microsecond*

In [28]:
birthday = datetime.datetime(1983,9,9,17)
print(birthday)

1983-09-09 17:00:00


*We can also create a datetime object from a string by defining the format the string represents*

In [29]:
dt = datetime.datetime.strptime("20000412","%Y%m%d")
print(dt)

2000-04-12 00:00:00


**We can now complete our new data set**

In [30]:
print(re.search(pDate,newData[0]['src']).groups(1)[0])
print(re.search(pDataset,newData[0]['src']).groups(1)[0])

19781026
nsidc0051


In [31]:
for d in newData:
    src = d['src']
    date = re.search(pDate,src).groups(1)[0]
    dataset = re.search(pDataset,src).groups(1)[0]
    d['Date'] = str(datetime.datetime.strptime(date,"%Y%m%d"))
    d['Dataset'] = dataset
    del d['src']

In [32]:
for d in newData[0:9]:
    print(d)
for d in newData[-10:]:
    print(d)

{'Extent': 10.231, 'hemisphere': 'north', 'Date': '1978-10-26 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.42, 'hemisphere': 'north', 'Date': '1978-10-28 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.557, 'hemisphere': 'north', 'Date': '1978-10-30 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.67, 'hemisphere': 'north', 'Date': '1978-11-01 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.777, 'hemisphere': 'north', 'Date': '1978-11-03 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.968, 'hemisphere': 'north', 'Date': '1978-11-05 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 11.08, 'hemisphere': 'north', 'Date': '1978-11-07 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 11.189, 'hemisphere': 'north', 'Date': '1978-11-09 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 11.314, 'hemisphere': 'north', 'Date': '1978-11-11 00:00:00', 'Dataset': 'nsidc0051'}
{'Extent': 10.078, 'hemisphere': 'south', 'Date': '2019-05-29 00:00:00', 'Dataset': 'nsidc0081'}
{'Extent': 10.219, 'hemisphere': 

In [33]:
fieldnames = [key for key in newData[0].keys()]
print(fieldnames)

['Extent', 'hemisphere', 'Date', 'Dataset']


*We want to re-order the list so the columns will be: Date,Dataset,hemisphere,Extent*

In [34]:
order = [2,3,1,0]
fieldnames = [fieldnames[i] for i in order]
print(fieldnames)

['Date', 'Dataset', 'hemisphere', 'Extent']


In [35]:
with open('sea-ice-fixed.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for entry in newData:
        writer.writerow(entry)